In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms

# Dataset

In [2]:
class SquareDataset(Dataset):
    def __init__(self, size):
        self.size = size
        self.X = torch.randint(255, (size, 9), dtype=torch.float)

        real_w = torch.tensor([[1,1,1,0,0,0,0,0,0],
                               [0,0,0,1,1,1,0,0,0],
                               [0,0,0,0,0,0,1,1,1]], 
                               dtype=torch.float)

        y = torch.argmax(self.X.mm(real_w.t()), 1)
        
        self.Y = torch.zeros(size, 3, dtype=torch.float) \
                      .scatter_(1, y.view(-1, 1), 1)

    def __getitem__(self, index):
        return (self.X[index], self.Y[index])

    def __len__(self):
        return self.size

In [3]:
squares = SquareDataset(256)
print(squares[34])
print(squares[254])
print(squares[25])

(tensor([ 54., 182.,  47., 142., 200., 197., 220., 215.,  33.]), tensor([0., 1., 0.]))
(tensor([198., 171.,  26., 140.,  28.,   9., 205.,  48., 113.]), tensor([1., 0., 0.]))
(tensor([ 64.,   7., 167.,   4.,   9., 160., 169., 113., 214.]), tensor([0., 0., 1.]))


In [4]:
dataloader = DataLoader(squares, batch_size=5)

for batch, (X, Y) in enumerate(dataloader):
    print(X, '\n\n', Y)
    break

tensor([[152., 127., 155., 219.,  81., 140., 112.,  77., 102.],
        [ 77.,  58., 228., 164., 229., 155., 111., 223., 141.],
        [106., 250.,  87.,  62., 105., 254.,   0., 210., 136.],
        [190., 108., 134., 204., 145., 251., 146., 171.,  99.],
        [ 88.,  36., 190., 108., 122.,   4., 231.,  22.,  70.]]) 

 tensor([[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


# Digits
Transforms!

In [5]:
digits = datasets.MNIST('data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Lambda(lambda x: x.view(28*28))
                    ]),
                    target_transform=transforms.Compose([
                        transforms.Lambda(lambda y: 
                                          torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    ])
                 )

In [6]:
dataloader = DataLoader(digits, batch_size=10, shuffle=True)

for batch, (X, Y) in enumerate(dataloader):
    print(X, '\n\n', Y)
    break

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) 

 tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])
